In [51]:
prog1 = """
a(1).
#program acid( k ).
b( k ).
c( X,k ) :- a( X ).
#program base.
a (2).
"""

In [52]:
import clingo
from clingo.control import Control

ctl = Control("0")
ctl.add("base", [], prog1)
ctl.ground([("base", [])])
ctl.ground([("acid", [])])
ctl.ground([("acid", [clingo.Number(1)])])
ctl.ground([("acid", [clingo.Number(2)])])
ctl.solve(on_model=print)

a(1) a(2) b(1) b(2) c(1,1) c(2,1) c(1,2) c(2,2)


SolveResult(5)

In [53]:
from clingo.symbol import Number, Function
from clingo.control import Control

prog = """
#program base.
a(1;2).

#program acid(k).
b(k).
c(X,k) :- a(X).

#external d( X,k ) : c( X,k ).
e( X,k ) :- d( X,k ).

#program graph(e,weight).
edge(e,weight).


"""

ctl = Control("0")
ctl.add("base", [], prog)
ctl.ground([("base", [])])
#ctl.add("acid", ["k"], acid)
ctl.ground([("acid", [Number(42)])])
ctl.assign_external(Function("d", [Number(2), Number(42)]), True)

ctl.ground([("graph", [Function("nodes", [Number(1), Number(2)]), Number(17)])])

ctl.solve(on_model=print)


a(1) a(2) b(42) edge(nodes(1,2),17) c(1,42) c(2,42) d(2,42) e(2,42)


SolveResult(5)

In [54]:
class Context:
    def id(self, x):
        return x

    def seq(self, x, y):
        return [x, y]

    def next(self, s):
        return [Number(s.number)]  #, Number(4), Number(5), Number(s)]


incr = """
pos(1,0). % actual position(Node,Timepoint)
nat(15).
#program step(n).
1{next(@next(Now),n+1)}1 :- pos(Now,n).

%pos(2,n).
pos(@next(X),n) :- nat(X).
"""

ctl = Control("0")
ctl.add("base", [], incr)
ctl.ground([("base", [])])
ctl.ground([("step", [Number(0)])], context=Context())

ctl.solve(on_model=print)

pos(1,0) pos(15,0) nat(15) next(1,1) next(15,1)


SolveResult(5)

### Propagator

In [55]:
inst = """
a.
"""

#prop = GPropagator()
ctl = Control("0")
#ctl.register_propagator(prop)
ctl.add("base", [], inst)
ctl.ground([("base", [])])
ctl.solve(on_model=print)

a


SolveResult(5)

In [56]:
p_a = clingo.Function("p", [clingo.Function("A")])
q_a = clingo.Function("q", [clingo.Function("A")])
q_1 = clingo.Function("q", [clingo.Number(1)])


In [57]:
from clingox.program import Program, ProgramObserver
from clingox.backend import SymbolicBackend

ctl = clingo.Control()
prg = Program()
ctl.register_observer(ProgramObserver(prg))

with SymbolicBackend(ctl.backend()) as b:
    b.add_rule(head = [p_a], pos_body = [q_a])
    b.add_rule(head= [q_1])

ctl.ground([("base", [])])
print(prg)
print()
ctl.solve(on_model=print)

q(1).
p(A) :- q(A).
__x3.

q(1)


SolveResult(1)